In [1]:
import pandas as pd
import socket
import requests
import dns.resolver

# Função para obter o IP de um domínio
def get_ip(domain):
    try:
        return socket.gethostbyname(domain)
    except Exception as e:
        return None

# Função para obter a localização do IP usando a API ipinfo.io
def get_location(ip):
    if ip:
        try:
            response = requests.get(f'https://ipinfo.io/{ip}/json')
            data = response.json()
            return {
                'city': data.get('city', ''),
                'region': data.get('region', ''),
                'country': data.get('country', ''),
                'org': data.get('org', ''),
                'postal': data.get('postal', ''),
                'timezone': data.get('timezone', ''),
                'hostname': data.get('hostname', ''),
                'latitude': '',
                'longitude': '',
            }
        except Exception as e:
            return {
                'city': '',
                'region': '',
                'country': '',
                'org': '',
                'postal': '',
                'timezone': '',
                'hostname': '',
                'latitude': '',
                'longitude': '',
            }
    return {
        'city': '',
        'region': '',
        'country': '',
        'org': '',
        'postal': '',
        'timezone': '',
        'hostname': '',
        'latitude': '',
        'longitude': '',
    }

# Função para obter o DNS reverso de um IP
def get_reverse_dns(ip):
    if ip:
        try:
            return socket.gethostbyaddr(ip)[0]
        except Exception as e:
            return None
    return None

# Função para obter o número de registros DNS de um IP
def get_dns_count(ip):
    if ip:
        try:
            answers = dns.resolver.resolve(ip, 'A')
            return len(answers)
        except Exception as e:
            return 0
    return 0

# Carregar o dataset, ignorando linhas problemáticas
df = pd.read_csv('ps.csv', on_bad_lines='skip', delimiter=';')

# Adicionar colunas para armazenar IP, DNS, Reverse DNS e número de DNS
df['IP'] = df['URL'].apply(lambda url: get_ip(url.split('//')[1]))
df['DNS'] = df['URL'].apply(lambda url: url.split('//')[1])
df['Reverse DNS'] = df['IP'].apply(get_reverse_dns)
df['n_dns'] = df['IP'].apply(get_dns_count)

# Obter informações de localização e adicionar colunas separadas
location_data = df['IP'].apply(get_location)
df['City'] = location_data.apply(lambda loc: loc['city'])
df['Region'] = location_data.apply(lambda loc: loc['region'])
df['Country'] = location_data.apply(lambda loc: loc['country'])
df['Org'] = location_data.apply(lambda loc: loc['org'])
df['Postal'] = location_data.apply(lambda loc: loc['postal'])
df['Timezone'] = location_data.apply(lambda loc: loc['timezone'])
df['Hostname'] = location_data.apply(lambda loc: loc['hostname'])

# Separação da chave 'loc' em latitude e longitude
for i, loc in enumerate(location_data):
    if 'loc' in loc:
        latitude, longitude = loc['loc'].split(',')
        df.at[i, 'Latitude'] = latitude.strip()
        df.at[i, 'Longitude'] = longitude.strip()

# Salvar o resultado em um novo arquivo CSV
df.to_csv('ps_with_ip_location_dns.csv', index=False)

# Exibir o DataFrame
print(df.head())


                                  URL  URLLength                      Domain  \
0    https://www.southbankmosaics.com         31    www.southbankmosaics.com   
1            https://www.uni-mainz.de         23            www.uni-mainz.de   
2      https://www.voicefmradio.co.uk         29      www.voicefmradio.co.uk   
3         https://www.sfnmjournal.com         26         www.sfnmjournal.com   
4  https://www.rewildingargentina.org         33  www.rewildingargentina.org   

   DomainLength  IsDomainIP  TLD  URLSimilarityIndex  CharContinuationRate  \
0            24           0  com                 100              1.000000   
1            16           0   de                 100              0.666667   
2            22           0   uk                 100              0.866667   
3            19           0  com                 100              1.000000   
4            26           0  org                 100              1.000000   

   TLDLegitimateProb  URLCharProb  ...            